In [1]:
import json 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model,load_model,save_model
import scipy.linalg as la
import plotly.io as pio
import dash_bootstrap_components as dbc
pio.renderers.default = 'browser'

In [2]:
with open('./stress_data/parameter_amplitude.json', 'r') as file:
    parameter = json.load(file)
X_train_mean=parameter["mean_amplitude"]
X_train_std=parameter["STD_amplitude"]

In [3]:
X_train_mean=np.array(X_train_mean)
X_train_std=np.array(X_train_std)

In [4]:
X_train_mean=X_train_mean.reshape(1,1,5)
X_train_std=X_train_std.reshape(1,1,5)

In [5]:
node_location=pd.read_csv("./stress_data/node_location_amplitude.csv")
node_location.index=node_location["node_number"]
node_location=node_location.drop("node_number",axis=1)

In [6]:
model_amplitude = load_model('./MODEL_SAVE/model_cnn_amplitude.keras')
model_mean = load_model('./MODEL_SAVE/model_cnn_mean.keras')

In [7]:
def tet_to_tris(element):
    tris = [[None for j in range(3)] for i in range(4 * len(element))]
    for i in range(len(element)):
        j = 4 * i
        n0 = element[i][0]
        n1 = element[i][1]
        n2 = element[i][2]
        n3 = element[i][3]

        # Triangle-0
        tris[j][0] = n0
        tris[j][1] = n1
        tris[j][2] = n3

        # Triangle-1
        tris[j + 1][0] = n1
        tris[j + 1][1] = n2
        tris[j + 1][2] = n3

        # Triangle-2
        tris[j + 2][0] = n0
        tris[j + 2][1] = n3
        tris[j + 2][2] = n2

        # Triangle-3
        tris[j + 3][0] = n0
        tris[j + 3][1] = n2
        tris[j + 3][2] = n1

    return tris

In [8]:
with open('./CONT_PLOT/CONROD_NODE.txt', 'r') as f:
    node1 = np.array([[float(num) for num in line.split(',')] for line in f])

numnode = len(node1)
node = np.zeros((numnode, 3))
for i in range(0, numnode):
    node[i, :] = node1[i, 1:4]

# Load element data
with open('./CONT_PLOT/CONROD_ELEM.txt', 'r') as f:
    element = np.array([[int(num) for num in line.split(',')] for line in f])

numelem = len(element)
element2 = np.zeros((numelem, 10))
for el in range(0, numelem):
    econ = element[el, 1:11]
    element2[el, :] = econ - 1

element = element2.astype('int')

In [9]:
def haigh_diagram_data(ultimate_tensile_strength, fatigue_limit, num_points=100):
    mean_stress = np.linspace(ultimate_tensile_strength, 0, num_points)
    stress_range = np.linspace(0, fatigue_limit, num_points)
    return mean_stress, stress_range

def haigh_diagram_yeild(yield_limit, num_points=100):
    yeild_x = np.linspace(yield_limit, 0, num_points)
    yeild_y = np.linspace(0, yield_limit, num_points)
    return yeild_x, yeild_y

def haigh_diagram_yeild_ng(yield_limit, num_points=100):
    yeild_x = -1 * (np.linspace(yield_limit, 0, num_points))
    yeild_y = np.linspace(0, yield_limit, num_points)
    return yeild_x, yeild_y

def haigh_diagram_data_ap(fatigue_limit, num_points=25):
    stress_range = np.linspace(fatigue_limit, fatigue_limit, 25)
    mean_stress = np.linspace(-200, 0, 25)
    return mean_stress, stress_range

# Generate Haigh diagram data
ultimate_tensile_strength = 850  # MPa
fatigue_limit = 450  # MPa
yield_limit = 650  # Mpa

mean_stress, stress_range = haigh_diagram_data(ultimate_tensile_strength, fatigue_limit)
yeild_x, yeild_y = haigh_diagram_yeild(yield_limit)
yeild_x_n, yeild_y_n = haigh_diagram_yeild_ng(yield_limit)
mean_stress_ag, stress_range_ag = haigh_diagram_data_ap(fatigue_limit, 25)

In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import datetime
import json

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define app layout
app.layout = html.Div([
    dbc.Container([
        dbc.Row([
            dbc.Col(html.H1("Dashboard", className="text-center text-primary mb-4"), width=12)
        ]),
        dbc.Row([
            dbc.Col(dcc.Graph(id='PFP', style={'height': '600px', 'width': '100%'}), width=6),
            dbc.Col(dcc.Graph(id='RPM', style={'height': '600px', 'width': '100%'}), width=6),
        ]),
        dbc.Row([
            dbc.Col(dcc.Graph(id='3d-visualization', style={'height': '600px', 'width': '100%'}), width=6),
            dbc.Col(dcc.Graph(id='haigh-diagram', style={'height': '600px', 'width': '100%'}), width=6),
        ]),
        dcc.Interval(
            id='interval-component',
            interval=2 * 1000,  # Update every 2 seconds
            n_intervals=0
        ),
        dcc.Store(id='store-data1', data={'x': [], 'y': []}),  # Store previous data for graph 1
        dcc.Store(id='store-data2', data={'x': [], 'y': []}),  # Store previous data for graph 2
    ], fluid=True)
])

@app.callback(
    [Output('PFP', 'figure'), Output('store-data1', 'data'),
     Output('RPM', 'figure'), Output('store-data2', 'data'),
    Output('3d-visualization', 'figure'), Output('haigh-diagram', 'figure')],
    [Input('interval-component', 'n_intervals')],
    [State('store-data1', 'data'), State('store-data2', 'data')]
)
def update_graph_live(n, stored_data1, stored_data2):
    current_time = datetime.datetime.now()

    # Update stored data for graph 1
    stored_data1['x'].append(current_time)
    pressure=np.arange(5,30,20)
    gas=np.random.choice(pressure)
    stored_data1['y'].append(gas)
    node_location["gas_load"]= node_location.apply(lambda x: gas*1000000, axis=1)


    # Update stored data for graph 2 (different parameter)
    stored_data2['x'].append(current_time)
    RPM=np.arange(500,1500,20)
    inertia_load=np.random.choice(RPM)
    stored_data2['y'].append(inertia_load)  # Different range for demonstration
    node_location["inertia_load"]=node_location.apply(lambda x:inertia_load, axis=1)
    X=node_location[["X","Y","Z","gas_load","inertia_load"]].to_numpy()
    X=X.reshape(1,len(node_location),5)
    X_train = (X - X_train_mean) / X_train_std
    amplitude=model_amplitude.predict(X_train,verbose=0)
    mean=model_mean.predict(X_train,verbose=0)
    Data = amplitude.flatten()
    Data = np.concatenate((Data, np.zeros(numnode - len(Data))))
    
    tris = tet_to_tris(element)
    numtri = len(tris)
    idi = np.zeros(numtri)
    idj = np.zeros(numtri)
    idk = np.zeros(numtri)

    for i in range(0, numtri):
        idi[i] = tris[i][0]
        idj[i] = tris[i][1]
        idk[i] = tris[i][2]

    idi = idi.astype('int')
    idj = idj.astype('int')
    idk = idk.astype('int')
    ind = np.argmax(amplitude)

    fig = go.Figure(data=[
        go.Mesh3d(
            x=node[:, 0],
            y=node[:, 1],
            z=node[:, 2],
            colorbar_title='Amplitude_stress',
            colorscale='jet',
            intensity=Data,  # Use padded Data
            i=idi,
            j=idj,
            k=idk,
            cmin=0,
            cmax=np.max(Data),
            name='y',
            opacity=1,
        )
    ])
    
    fig.update_layout(
        scene=dict(
            annotations=[
                dict(
                    x=str(node[ind, 0]),
                    y=str(node[ind, 1]),
                    z=str(node[ind, 2]),
                    text="max: " + str(np.round(amplitude[0,ind], 3)),  # Adjust annotation as per your prediction format
                    textangle=0,
                    ax=0,
                    ay=-75,
                    font=dict(
                        color="black",
                        size=12
                    ),
                    arrowcolor="black",
                    arrowsize=3,
                    arrowwidth=1,
                    arrowhead=1),
            ]
        ),
        scene_aspectmode='data'
    )
    haigh_diagram_fig = go.Figure()
    haigh_diagram_fig.add_trace(go.Scatter(x=mean_stress, y=stress_range, mode='lines', name="ult_Fatigue",
                                           line=dict(color="red", dash="dash")))
    haigh_diagram_fig.add_trace(go.Scatter(x=yeild_x, y=yeild_y, mode='lines', name="Fatigue-Yield",
                                           line=dict(color="blue", dash="dash")))
    haigh_diagram_fig.add_trace(go.Scatter(x=yeild_x_n, y=yeild_y_n, mode='lines', name="Yield",
                                           line=dict(color="blue", dash="dash")))
    haigh_diagram_fig.add_trace(go.Scatter(x=mean_stress_ag, y=stress_range_ag, mode='lines', name="Yield",
                                           line=dict(color="red", dash="dash")))
    haigh_diagram_fig.add_trace(go.Scatter(x=amplitude[0,ind], y=mean[0,ind], mode='markers',
                                           marker=dict(color='red', size=10, symbol='star'),
                                           name='CNN Prediction'))
    haigh_diagram_fig.update_layout(title='Haigh Diagram for 42CrMo4',
        xaxis_title='Mean Stress (MPa)',
        yaxis_title='Stress amplitude (MPa)',
        xaxis=dict(range=[-ultimate_tensile_strength * 1.1, ultimate_tensile_strength * 1.1]),
        yaxis=dict(range=[0, ultimate_tensile_strength]),
        showlegend=True)

    data1 = go.Scatter(
        x=stored_data1['x'],
        y=stored_data1['y'],
        mode='lines+markers'
    )

    data2 = go.Scatter(
        x=stored_data2['x'],
        y=stored_data2['y'],
        mode='lines+markers'
    )

    figure1 = {
        'data': [data1],
        'layout': go.Layout(
            xaxis=dict(title='Time'),
            yaxis=dict(title='PFP', range=[0, 35]),
            title='PFP Values (Updated Every 2 Seconds)'
        )
    }

    figure2 = {
        'data': [data2],
        'layout': go.Layout(
            xaxis=dict(title='Time'),
            yaxis=dict(title='RPM', range=[0, 1500]),
            title='RPM (Updated Every 2 Seconds)'
        )
    }

    return figure1, stored_data1, figure2, stored_data2,fig,haigh_diagram_fig

if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\frame.py:3980, in DataFrame.__setitem__(
    self=                      X             Y          Z...  5000000          1440

[10301 rows x 5 columns],
    key='inertia_load',
    value=node_number
1        900
2        900
3        9...0    900
10301    900
Length: 10301, dtype: int32
)
   3977     self._setitem_array([key], value)
   3978 else:
   3979     # set column
-> 3980     self._set_item(key, value)
        key = 'inertia_load'
        self =                       X             Y          Z  gas_load  inertia_load
node_number                                                             
1            -75.340533 -8.978735e+01 -31.900556  25000000           920
2            -74.673384 -8.899227e+01 -34.346660  25000000           920
3            -86.500000  0.000000e+00 -

---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File C:\ProgramData\anaconda3\Lib\site-packages\pandas\core\frame.py:3980, in DataFrame.__setitem__(
    self=                      X             Y          Z... 25000000           580

[10301 rows x 5 columns],
    key='inertia_load',
    value=node_number
1        1180
2        1180
3       ...   1180
10301    1180
Length: 10301, dtype: int32
)
   3977     self._setitem_array([key], value)
   3978 else:
   3979     # set column
-> 3980     self._set_item(key, value)
        key = 'inertia_load'
        self =                       X             Y          Z  gas_load  inertia_load
node_number                                                             
1            -75.340533 -8.978735e+01 -31.900556  25000000           580
2            -74.673384 -8.899227e+01 -34.346660  25000000           580
3            -86.500000  0.000000e+00 -